In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from tensorflow import keras
from keras.models import Sequential,load_model
from keras.layers import Dense,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Dropout,MaxPooling2D,Activation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator as Imgen

from PIL import Image
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
train_df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")

In [ ]:
train_df.shape

In [ ]:
train_df.labels.value_counts()

In [ ]:
sample = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample.head()

**Image Data Gen**

In [ ]:
datagen = Imgen(preprocessing_function=keras.applications.xception.preprocess_input,
                 rotation_range=4,
                  shear_range=0.2,
                  zoom_range=0.2,
                  horizontal_flip=True,
                  validation_split=0.2
                 )

**Data Generation**

In [ ]:
train_ds = datagen.flow_from_dataframe(
    train_df,
    directory = '../input/resized-plant2021/img_sz_256', 
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123,
)


val_ds = datagen.flow_from_dataframe(
    train_df,
    directory = '../input/resized-plant2021/img_sz_256',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123,
)

In [ ]:
a = train_ds.class_indices
class_names = list(a.keys())
class_names

In [ ]:
x,y = next(train_ds)
x.shape

**Visualize some images**

In [ ]:
#plot function
def plot_images(img,labels):
    plt.figure(figsize=(15,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(img[i])
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')

In [ ]:
plot_images(x,y)

## **MODEL**

In [ ]:
model = Sequential([
    
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(128,activation='relu'),
    Dropout(0.5),
    
    Dense(64,activation='relu'),
    Dropout(0.5),
    
    Dense(12,activation='softmax')
    
])

model.summary()

In [ ]:
keras.utils.plot_model(model,
                      show_shapes=True,
                      show_dtype=True,
                      show_layer_names=True)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
my_calls = [keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2),
            keras.callbacks.ModelCheckpoint("Model_xcp.h5",verbose=1,save_best_only=True)]

**Train**

In [ ]:
hist = model.fit(train_ds,epochs=25,validation_data=val_ds,callbacks=my_calls)

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()

In [ ]:
sample.head()

In [ ]:
test_gen = Imgen(preprocessing_function=keras.applications.xception.preprocess_input)

test_ds = test_gen.flow_from_dataframe(sample,
                                      directory='../input/plant-pathology-2021-fgvc8/test_images',
                                      x_col = 'image',
                                       y_col = None,
                                       class_mode=None,
                                       target_size=(224,224)
                                      )

In [ ]:
pred = model.predict(test_ds)
pred

In [ ]:
pred = [np.argmax(i) for i in pred]
for i in pred:
    print(class_names[i])